<a href="https://colab.research.google.com/github/hung2706/-a/blob/main/%C4%91%E1%BB%93_%C3%A1n_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#TIỀN XỬ LÍ DỮ LIỆU


from google.colab import files
uploaded = files.upload()


Saving 2.xlsx to 2.xlsx


In [10]:
import pandas as pd
import tensorflow as tf

df = pd.read_excel("2.xlsx")
df.columns = ['input', 'response']  # Đổi tên cột để dễ dùng về sau
df.head()

import re
# Hàm làm sạch 1 câu
def clean_text(text):
    text = text.lower()  # chuyển về chữ thường
    text = re.sub(r"[^\w\sàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]", " ", text)
    text = re.sub(r"\s+", " ", text)  # loại bỏ khoảng trắng thừa
    return text.strip()
# Áp dụng cho cả input và response
df['input'] = df['input'].apply(clean_text)
df['response'] = df['response'].apply(clean_text)
# Thêm token <start> và <end> cho câu trả lời
df['response'] = df['response'].apply(lambda x: '<start> ' + x + ' <end>')

# Kiểm tra thử vài dòng
for i in range(3):
    print(f"Q: {df['input'][i]}")
    print(f"A: {df['response'][i]}")
    print("---")



FileNotFoundError: [Errno 2] No such file or directory: '2.xlsx'

In [5]:
#TOKENIZE

from tensorflow.keras.preprocessing.text import Tokenizer

# 1. Tạo tokenizer CHUNG cho cả input và response
shared_tokenizer = Tokenizer(filters='', oov_token='<OOV>')

# 2. Huấn luyện tokenizer trên toàn bộ dữ liệu (cả input và response)
all_texts = df['input'].tolist() + df['response'].str.replace('<start>|<end>', '').tolist()
shared_tokenizer.fit_on_texts(all_texts)

# 3. Mã hóa câu
encoder_input_sequences = shared_tokenizer.texts_to_sequences(df['input'])

# Xử lý response: bỏ <start>/<end> cũ, thêm lại bằng tay để đảm bảo nhất quán
temp_sequences = shared_tokenizer.texts_to_sequences(df['response'].str.replace('<start>|<end>', ''))
decoder_input_sequences = [
    [shared_tokenizer.word_index['<start>']] + seq + [shared_tokenizer.word_index['<end>']]
    for seq in temp_sequences
]

# 4. In thử để kiểm tra
for i in range(3):
    print(f"Q{i+1} (chuỗi số):", encoder_input_sequences[i])
    print(f"A{i+1} (chuỗi số):", decoder_input_sequences[i])
    print("---")

# 5. Xem từ điển từ → số (nếu bạn muốn)
print("Từ điển shared_tokenizer (input + response):", shared_tokenizer.word_index)

# Tính MAX_DECODER_LEN dựa trên độ dài thực tế của câu trả lời
MAX_DECODER_LEN = max(len(seq) for seq in decoder_input_sequences)  # Giữ nguyên độ dài tự nhiên
print("Độ dài tối đa thực tế:", MAX_DECODER_LEN)  # Kiểm tra xem giá trị hợp lý không (nên < 20)

Q1 (chuỗi số): [67, 15]
A1 (chuỗi số): [2, 2, 15, 4, 4, 150, 42, 10, 3, 3]
---
Q2 (chuỗi số): [15, 31, 44]
A2 (chuỗi số): [2, 2, 15, 31, 44, 101, 4, 26, 24, 151, 152, 3, 3]
---
Q3 (chuỗi số): [15, 31, 32]
A3 (chuỗi số): [2, 2, 15, 31, 32, 4, 153, 100, 32, 28, 3, 3]
---
Từ điển shared_tokenizer (input + response): {'<OOV>': 1, '<start>': 2, '<end>': 3, 'bạn': 4, 'là': 5, 'có': 6, 'không': 7, 'mình': 8, 'bằng': 9, 'gì': 10, '2': 11, 'học': 12, 'bao': 13, 'nhiêu': 14, 'chào': 15, 'nhé': 16, 'đâu': 17, 'hôm': 18, 'nay': 19, 'trời': 20, 'biết': 21, 'thể': 22, 'mấy': 23, 'ngày': 24, '5': 25, 'một': 26, 'từ': 27, 'chưa': 28, 'được': 29, 'nè': 30, 'buổi': 31, 'tối': 32, 'đang': 33, 'cộng': 34, '3': 35, 'nhân': 36, '4': 37, 'python': 38, 'trong': 39, 'máy': 40, 'đi': 41, 'giúp': 42, 'của': 43, 'sáng': 44, 'nhiều': 45, 'giờ': 46, 'độ': 47, 'đẹp': 48, 'trừ': 49, '6': 50, 'chia': 51, 'trình': 52, 'đầu': 53, 'tính': 54, 'giải': 55, 'x': 56, 'hát': 57, 'chuyện': 58, 'sống': 59, 'rất': 60, 'để': 61, 

In [7]:
  #PADDING

from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_ENCODER_LEN = 15  # Độ dài tối đa cho câu hỏi
MAX_DECODER_LEN = 20  # Độ dài tối đa cho câu trả lời
# Dịch chuyển decoder_input_sequences sang trái 1 bước để tạo decoder_target_sequences
decoder_target_sequences = [seq[1:] for seq in decoder_input_sequences]

# Để đồng bộ độ dài, bạn có thể thêm 0 ở cuối mỗi câu (vì đã bỏ 1 phần tử đầu)
decoder_target_sequences = [seq + [0] for seq in decoder_target_sequences]

# Padding cho encoder input (câu hỏi)
encoder_input_padded = pad_sequences(encoder_input_sequences, maxlen=MAX_ENCODER_LEN, padding='post', truncating='post')

# Padding cho decoder input (câu trả lời đầu vào)
decoder_input_padded = pad_sequences(decoder_input_sequences, maxlen=MAX_DECODER_LEN, padding='post', truncating='post')

# Padding cho decoder output (dịch chuyển một bước để so sánh khi huấn luyện)
decoder_target_padded = pad_sequences(decoder_target_sequences, maxlen=MAX_DECODER_LEN, padding='post', truncating='post')

import numpy as np
decoder_target_padded = np.expand_dims(decoder_target_padded, -1)

print("encoder_input_padded.shape:", encoder_input_padded.shape)
print("decoder_input_padded.shape:", decoder_input_padded.shape)
print("decoder_target_padded.shape:", decoder_target_padded.shape)


encoder_input_padded.shape: (400, 15)
decoder_input_padded.shape: (400, 20)
decoder_target_padded.shape: (400, 20, 1)


TIẾP TỤC Ở ĐÂY :::


In [11]:
from tensorflow.keras.callbacks import Callback
#EN+DECODER
DROPOUT_RATE = 0.2
EMBEDDING_DIM = 64  # Giảm do dataset nhỏ
LATENT_DIM = 128

from tensorflow.keras.layers import Dot, Activation
from tensorflow.keras.layers import AdditiveAttention, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

# Số lượng từ trong tokenizer
VOCAB_SIZE = len(shared_tokenizer.word_index) + 1
# Siêu tham số
EMBEDDING_DIM = 128
LATENT_DIM = 256

# ENCODER
encoder_inputs = Input(shape=(MAX_ENCODER_LEN,))
encoder_embedding = Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM)(encoder_inputs)
encoder_lstm_outputs, state_h, state_c = LSTM(
    LATENT_DIM,
    return_sequences=True,
    return_state=True,
    dropout=DROPOUT_RATE  # <== THÊM DROPOUT
)(encoder_embedding)

# DECODER (phần training)
decoder_inputs = Input(shape=(MAX_DECODER_LEN,))
decoder_embedding = Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM)(decoder_inputs)
decoder_lstm_outputs, _, _ = LSTM(
    LATENT_DIM,
    return_sequences=True,
    return_state=True,
    dropout=DROPOUT_RATE  # <== THÊM DROPOUT
)(decoder_embedding, initial_state=[state_h, state_c])

# ATTENTION
attention_layer = AdditiveAttention()([decoder_lstm_outputs, encoder_lstm_outputs])
context = attention_layer
decoder_combined_context = Concatenate(axis=-1)([context, decoder_lstm_outputs])

# COMBINE CONTEXT + DECODER
decoder_combined_context = Concatenate(axis=-1)([context, decoder_lstm_outputs])
decoder_outputs = Dense(VOCAB_SIZE, activation='softmax')(decoder_combined_context)

class TeacherForcingScheduler(Callback):
    def __init__(self, initial_prob=1.0):
        self.prob = initial_prob

    def on_epoch_begin(self, epoch, logs=None):
        # Giảm dần teacher forcing từ 100% -> 50% sau 10 epochs
        self.prob = max(0.5, 1.0 - epoch/10)
        print(f"\nEpoch {epoch+1}: Teacher Forcing Prob = {self.prob:.2f}")


# MODEL
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 15, 128)   │     32,000 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_5         │ (None, 20, 128)   │     32,000 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, 15, 256), │    394,240 │ embedding_4[0][0] │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ [(None, 20, 256), │    394,240 │ embedding_5[0][0… │
│                     │ (None, 256),      │            │ lstm_4[0][1],     │
│                     │ (None, 256)]      │            │ lstm_4[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ additive_attention… │ (None, 20, 256)   │        256 │ lstm_5[0][0],     │
│ (AdditiveAttention) │                   │            │ lstm_4[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 20, 512)   │          0 │ additive_attenti… │
│ (Concatenate)       │                   │            │ lstm_5[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 20, 250)   │    128,250 │ concatenate_3[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 980,986 (3.74 MB)

 Trainable params: 980,986 (3.74 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
callbacks = [TeacherForcingScheduler()]
model.fit([encoder_input_padded, decoder_input_padded], decoder_target_padded,
          batch_size=32,
          epochs=50,
          validation_split=0.1,
          callbacks=callbacks)


Epoch 1: Teacher Forcing Prob = 1.00
Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 249ms/step - accuracy: 0.3562 - loss: 4.8893 - val_accuracy: 0.5288 - val_loss: 2.6748

Epoch 2: Teacher Forcing Prob = 0.90
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 189ms/step - accuracy: 0.5500 - loss: 2.4823 - val_accuracy: 0.5500 - val_loss: 2.3078

Epoch 3: Teacher Forcing Prob = 0.80
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 182ms/step - accuracy: 0.5745 - loss: 2.1997 - val_accuracy: 0.5800 - val_loss: 2.1280

Epoch 4: Teacher Forcing Prob = 0.70
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 283ms/step - accuracy: 0.5978 - loss: 2.0474 - val_accuracy: 0.6075 - val_loss: 1.9988

Epoch 5: Teacher Forcing Prob = 0.60
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 188ms/step - accuracy: 0.6158 - loss: 1.9315 - val_accuracy: 0.6138 - val_loss: 1.9009

Epoch 6: Teacher Forcing Prob = 0.50
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 183ms/step - accuracy: 0.6182 - loss: 1.8788 - val_accuracy: 0.6237 - val_loss: 1.8001

Epo

In [9]:
# 1. TEST
callbacks = [TeacherForcingScheduler()]
model.fit([encoder_input_padded, decoder_input_padded], decoder_target_padded,
          batch_size=32,
          epochs=50,
          validation_split=0.1,
          callbacks=callbacks)
history = model.fit(
    [encoder_input_padded, decoder_input_padded],
    decoder_target_padded,
    batch_size=32,
    epochs=50,
    validation_split=0.1,
    callbacks=callbacks
)

# 2. Xem các metrics đã lưu
print("Các metrics available:", history.history.keys())
# Output thường có: ['loss', 'accuracy', 'val_loss', 'val_accuracy']

# 3. Truy cập các chỉ số cụ thể
print("Training accuracy cuối cùng:", history.history['accuracy'][-1])
print("Validation loss cuối cùng:", history.history['val_loss'][-1])

NameError: name 'TeacherForcingScheduler' is not defined

In [18]:


# 1. Encoder model (giữ nguyên - đã đúng)
encoder_model = Model(encoder_inputs, [encoder_lstm_outputs, state_h, state_c])

# 2. DECODER MODEL mới - với đầy đủ attention
# Inputs cho inference
decoder_inputs_single = Input(shape=(1,))  # Mỗi lần chỉ sinh 1 token
decoder_state_input_h = Input(shape=(LATENT_DIM,))
decoder_state_input_c = Input(shape=(LATENT_DIM,))
decoder_hidden_state_input = Input(shape=(MAX_ENCODER_LEN, LATENT_DIM))  # encoder outputs

# Embedding layer (dùng lại từ training)
decoder_embedding_inf = Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM)(decoder_inputs_single)

# LSTM layer (dùng lại từ training)
decoder_lstm = LSTM(LATENT_DIM, return_sequences=True, return_state=True)
decoder_lstm_outputs_inf, state_h_inf, state_c_inf = decoder_lstm(
    decoder_embedding_inf,
    initial_state=[decoder_state_input_h, decoder_state_input_c]
)

# ATTENTION MECHANISM
attention_layer_inf = AdditiveAttention()(
    [decoder_lstm_outputs_inf, decoder_hidden_state_input, decoder_hidden_state_input]
)
context_inf = attention_layer_inf  # Shape sẽ là (None, 1, 256)
decoder_combined_context_inf = Concatenate(axis=-1)([context_inf, decoder_lstm_outputs_inf])

# COMBINE CONTEXT + DECODER (giống hệt training!)
context_inf = attention_layer_inf  # Bỏ phép nhân với decoder_hidden_state_input
decoder_combined_context_inf = Concatenate(axis=-1)([context_inf, decoder_lstm_outputs_inf])

# DENSE LAYER (giống hệt training!)
decoder_dense = Dense(VOCAB_SIZE, activation='softmax')
decoder_outputs_inf = decoder_dense(decoder_combined_context_inf)

# Tạo decoder model mới với architecture đúng
decoder_model = Model(
    [decoder_inputs_single, decoder_hidden_state_input, decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs_inf, state_h_inf, state_c_inf]
)

print("Decoder model rebuilt với attention mechanism!")
decoder_model.summary()

Decoder model rebuilt với attention mechanism!


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_26      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_11        │ (None, 1, 128)    │     32,000 │ input_layer_26[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_27      │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_28      │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_11 (LSTM)      │ [(None, 1, 256),  │    394,240 │ embedding_11[0][… │
│                     │ (None, 256),      │            │ input_layer_27[0… │
│                     │ (None, 256)]      │            │ input_layer_28[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_29      │ (None, 15, 256)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ additive_attention… │ (None, 1, 256)    │        256 │ lstm_11[0][0],    │
│ (AdditiveAttention) │                   │            │ input_layer_29[0… │
│                     │                   │            │ input_layer_29[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_11      │ (None, 1, 512)    │          0 │ additive_attenti… │
│ (Concatenate)       │                   │            │ lstm_11[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1, 250)    │    128,250 │ concatenate_11[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 554,746 (2.12 MB)

 Trainable params: 554,746 (2.12 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
def beam_decode(input_seq, beam_width=2, max_len=MAX_DECODER_LEN):
    # Bước 1: Encode input
    enc_outs, state_h, state_c = encoder_model.predict(input_seq, verbose=0)

    # Bước 2: Khởi tạo beam (sequence, score, hidden states)
    beams = [([shared_tokenizer.word_index['<start>']], 1.0, state_h, state_c)]

    for _ in range(max_len):
        candidates = []
        for seq, score, h, c in beams:
            # Dừng nếu gặp <end>
            if seq[-1] == shared_tokenizer.word_index['<end>']:
                candidates.append((seq, score, h, c))
                continue

            # Bước 3: Dự đoán từ tiếp theo
            target_seq = np.array([[seq[-1]]])
            output_tokens, h_new, c_new = decoder_model.predict(
                [target_seq, enc_outs, h, c], verbose=0)

            # Lấy top-k từ (beam_width)
            probs = output_tokens[0, -1, :]
            top_k = np.argsort(probs)[-beam_width:]  # Lấy indices của top-k

            for token in top_k:
                new_seq = seq + [token]
                new_score = score * probs[token]
                candidates.append((new_seq, new_score, h_new, c_new))

        # Bước 4: Chọn beam_width ứng viên tốt nhất
        beams = sorted(candidates, key=lambda x: x[1])[-beam_width:]

    # Bước 5: Trả về câu có điểm cao nhất (loại bỏ <start> và <end>)
    best_seq = beams[-1][0][1:]  # Bỏ <start>
    decoded_sentence = []
    for idx in best_seq:
        if idx == shared_tokenizer.word_index['<end>']:
            break
        decoded_sentence.append(shared_tokenizer.index_word.get(idx, '<OOV>'))

    return ' '.join(decoded_sentence)

In [20]:
user_input = input("Bạn: ")
user_input_seq = shared_tokenizer.texts_to_sequences([clean_text(user_input)])
user_input_padded = pad_sequences(user_input_seq, maxlen=MAX_ENCODER_LEN, padding='post')

# Dùng Beam Search thay vì decode_sequence cũ
print("Bot (Beam Search):", beam_decode(user_input_padded, beam_width=3))



Bạn: xin chào
Bot (Beam Search): thức thức thức thức thức thức thức thức thức thức thức thức thức thức dữ dữ dữ dữ dữ dữ


In [7]:
# Test nhanh chất lượng model hiện tại
test_samples = [
    ("xin chào", "chào bạn"),
    ("cảm ơn", "không có gì")
]

for input_text, true_output in test_samples:
    input_seq = shared_tokenizer.texts_to_sequences([input_text])
    padded = pad_sequences(input_seq, maxlen=MAX_ENCODER_LEN, padding='post')
    pred = beam_decode(padded)
    print(f"Input: {input_text} | Pred: {pred} | True: {true_output}")

NameError: name 'shared_tokenizer' is not defined